# The Theoretical Bedrock: Connecting the Dots

In classical computer science, we often perform operations directly on matrices, such as inversion, exponentiation, or polynomial transformations. In quantum computing, however, all physical operations must be unitary (represented by matrices U such that $U^\dagger U=\mathbb{1}$), which means they are reversible and preserve the norm of the state vector.

Most matrices of interest, such as Hamiltonians (which describe system energy), are Hermitian but not unitary. To solve this, modern quantum algorithms use Block Encodings to "embed" these non-unitary matrices into a larger unitary system, and then use Quantum Signal Processing (QSP) and its generalizations to manipulate their values.
## Block encoding 101
Block encoding is a foundational technique that allows a quantum computer to represent a general matrix A (where $∥A∥\leq1$) as the top-left block of a larger unitary matrix U.

The Mathematical Definition: A unitary U is an $(\alpha,m,\epsilon)$-block-encoding of A if, where $\alpha$ is a scaling factor, $m$ is the number of ancillary qubits, and $\epsilon$ is the precision. We essentially put a (possibly) non-unitary matrix in the top left block of a larger, unitary matrix. By projecting onto that top left subspace whe then essentially act exactly with the part of the matrix we wanted to apply, with the rest elements being disregarded.
$$\begin{pmatrix}
\frac{A}{\alpha} & *\\
* & *
\end{pmatrix}$$

In Qrisp, we can construct a block encoding in three ways:

- The ``.from_array`` constructs a BlockEncoding from a numpy array,

- ``.from_operator`` constructs a Blockencoding directly from a Hamiltonian constructed via QubitOperators, or FermionicOperators,

- and lastly, the ``.from_LCU`` constructs a BlockEncoding using the Linear Combination of Unitaries approach.

The first two options are rather self-explanatory, really... You (the user) provide the matrix and/or Hamiltonian you'd like to block encode, envoke the appropriate method for your desired input and, whoosh, Qrisp block encodes it for ya.




In [1]:
import numpy as np
from qrisp.block_encodings import BlockEncoding
A = np.array([[0,1,0,1],[1,0,0,0],[0,0,1,0],[1,0,0,0]])
B_A = BlockEncoding.from_array(A)

from qrisp.block_encodings import BlockEncoding
from qrisp.operators import X, Y
H = X(0)*X(1) + 0.2*Y(0)*Y(1)
B_H = BlockEncoding.from_operator(H)

Essentially the third approach, ``.from_LCU`` is the eptitome of block encoding. I mean, what it does is actually in the acronym: Linear Combination of Unitaries (or for the academic practicioners among you: $A=\sum_i\alpha_i U_i$).

If such a matrix $A$ is expressed as shown above (as a weighted sum of unitaries), the LCU algorithmic primitive uses three (well, technically two) oracles:

- PREPARE prepares (kinda obvious, huh?) a superposition of index states weighted of the square roots of the coefficients $\alpha_i$ in the ancilliary variable:$$\text{PREP}\ket{0}_a=\sum_{i=0}^{m-1}\sqrt{\frac{\alpha_i}{\alpha}}\ket{i}_a$$

- SELECT applies the corresponding unitary $U_i$ to the operand (target) variable, controlled by the ancilliary variable:$$\text{SEL}=\sum_{i=0}^{m-1}\ket{i}\bra{i}_a\otimes U_i$$,

- and finally PREPARE$^\dagger$, which then essentially unprepares the ancilliary variable.

We have sucessfully applied $A$ to our operand when the ancillary variable is measured in the $\ket{0}$ state.

For the visual learners, here is a figure that will (hopefully) tie everything you read so far together.

![Alt text](BE_bad_select.png)

We could also define a block encoding as a pair of unitaries $(U, G)$ ($G$ is another name for PREPARE, $U$ is another name for SELECT). Their action on quantum states can be captured by the following expressions (don't worry, we'll also use words about what this means immediately after).

$G$ simply prepares the ancilary variable in a superposition defined by the coefficients (or the weights) of the weighted sum of unitaries (LCU):$$G\ket{0}_{in_case}=\ket{G}_{in_case}.$$ Here we've denoted the ancillary variable as ``in_case``, short for "inner case".

$U$, on the other hand then selects "just the right amount" of each of the underlying unitaries in the weighted sum: $$U\ket{i}_{in_case}\ket{\psi}_{operand}=\ket{i}_{in_case}U_i\ket{\psi}_{operand}$$.

As seen in the picture above, this can be done via multiple comtrolled operations. In Qrisp, however, we use ``qswitch``, which is just another name for the select operator (multiplexor is another name found in literature for the same thing, btw). What's so special about ``qswitch``? Well, we've implemented a massively more efficient approach about how to apply such a (quantum) switch case (hence the name) based on balanced binary trees. For further information about this, please refer to the paper from Khattar and Gidney, [Rise of conditionally clean ancillae for efficient quantum circuit constructions](https://arxiv.org/pdf/2407.17966v1).

How are we feeling? I assume that the researchers in the field could be portrayed with this emoji (😎), while people coming from outside of quantum can feel a bit overwhelmed (🥲). Let's balance the target audiences by a relevant example that is the cornerstone of solivng partial differential equations, where we also explain how to use the ``.from_LCU`` method (went quite the tangent, huh? Glad you're still here - it'll be worth it!).

### Custom BlockEncoding - the discrete Laplacian

In this example, we construct a block encoding of the 1D discrete Laplacian operator with periodic boundary conditions using the ``.from_lcu`` method. The discrete Laplace operator is central in numerical physics and differential equations, representing the second derivative on a grid. 

Let's start by constructing the matrix in Numpy and prin it so that everyone is on the same page about what we'll be doing in this example.

In [2]:
import numpy as np

N = 8
I = np.eye(N)
L = 2 * I - np.eye(N, k=1) - np.eye(N, k=-1)
L[0, N-1] = -1
L[N-1, 0] = -1

print(L)

[[ 2. -1.  0.  0.  0.  0.  0. -1.]
 [-1.  2. -1.  0.  0.  0.  0.  0.]
 [ 0. -1.  2. -1.  0.  0.  0.  0.]
 [ 0.  0. -1.  2. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  2. -1.  0.  0.]
 [ 0.  0.  0.  0. -1.  2. -1.  0.]
 [ 0.  0.  0.  0.  0. -1.  2. -1.]
 [-1.  0.  0.  0.  0.  0. -1.  2.]]


This is exactly the finite-difference Laplacian with wrap-around endpoints.

We can, of course, use the ``.from_array`` method you're already a master of. Actually, let's do that real quick:

In [3]:
B_L = BlockEncoding.from_array(L)

Why even bother with ``.from_lcu`` then? Fair question. The answer hides in what's happening underneath, and the amount of **quantum resources** that these two methods use.

Woah, woah, woah?! Quantum resources? As in quanutm resources in Quantum Resource Estimation?

Yes.

"But isn't it currently a research endeavor about finding these exact resource estimates?!"

Yes. But... with Qrisp we made it as simple, clean, and qrispy (had to go for it), as calling the ``.resources`` method.

Let's see what do we get as an output:

In [4]:
from qrisp import QuantumFloat
def operand_prep():
    return QuantumFloat(2)
quantum_resources = B_L.resources(operand_prep)()
print(quantum_resources)

{'u3': 14, 'p': 4, 'gphase': 2, 'x': 9, 't_dg': 28, 'cy': 4, 'h': 14, 'cx': 67, 't': 21}K


Alright, the code runs with errors, but what does it actually do? You might also be wondering why this was called the future of programming quantum linear algebra - this doesn't seem all that special? Well, patience, young grasshopper... We've barely scratched the surface. Let's do some linear algebra now, i.e. calculate something and compare if the results we obtain for this quantum approach align with the results obtained from its classical cousin.

One neat feature of the BlockEncoding class is the 

## Qubitization

## Block encoding Chebyshev polynomials

## Quantum Signal Processing (QSP)

### Quantum Eigenvalue and Singular Value Transformation

### Generalized Quantum Signal Processing (GQSP)

## Conclusion

With the results, lemmas, theorems, and corollaries from the past two years now converging, the theoretical pieces fell into place. Stepping a step away and observing the mosaic with the final pieces added allowed for seeing the bigger picture and provide the common denominator of all these papers, the block encoding, as a standalone programming abstraction in Qrisp.

Since you've come this far down the theory rabbit-hole, the following tutorials will be a walk in the park, even if you're not that verse in programming. The syntax similar to numpy will make these concepts seem, at points, even too easy. The functionalities, seemingly simple pack quite a punch, I'll tell you that much.

See you on the other side (or rather part of this webSITE)... Yeah, yeah, I'll see myself take a break before continuting this pedagogical journey...

Before exploring the BlockEncoding class, we'll catch up on (just enough) theory to satisfy even the most rigorous academic. We'll dive into the math and connect the dots between a recent stream of publications (we're talking about papers who are all still less than a decade old!). 

This tutorial will cover:

- Block encodings: They allow us to embed a (possibly non-unitary) matrix $A$ (or a linear combination of them) into the top-left block of a larger unitary U. This makes $A$ "quantum-applicable". Once $U$ is implemented, repetaing this until the ancilla qubits are all measured in $\ket{0}$ effectivelly applies $A$ to a quantum state.

- Qubitization: Pairing a block encoded unitary with the reflection operator forms a walk operator acting on a small invariant subspace. This operator encodes the eigenvalues of $A$ as controllable rotations, forming the basis for block encoding Chebyshev polynomials, Childs-Kothari-Somma's algorithm, Hamiltonian simulation, and more. It's exactly these Chebyshevs that provide an optimal basis for bounded polynomial approximations. Take an inverse, for a popular example, to also entertain the QML fans among the readers and immediately plant the idea of solving linear systems and applying them as the basis for Quantum Support Vector Machines. I guess this kind of non-subliminal messaging is just messaging, huh?

- Quantum Signal Processing (QSP): A sequence of carefully chosen single‑qubit phase rotations that implements polynomial transformations $P(A)$ on your block encoding as precisely (based on $\epsilon$) and efficiently (based on $\kappa$) as you'd like (or have the resources for). We'll of course also mention what these two parameters represent (precision and condition number for the impatient ones among you).

- Generalized QSP (GQSP): Extends QSP to multiple operators and more flexible polynomial forms, handling the underlying phase calculations automatically. Enables advanced techniques such as spectral filtering (for ground‑state estimation) and quantum linear solvers. The (quantum) world is your oyster.
